In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

# Sample Data Creation (Increased size for demonstration)
data =pd.read_csv('flood_risk_dataset_india.csv')

# Create DataFrame
df = pd.DataFrame(data)
df=df.drop(columns=['Land Cover', 'Soil Type','Population Density','Elevation (m)','Infrastructure','River Discharge (m³/s)',], errors='ignore')


# Split features and target variable
X = df.drop('Flood Occurred', axis=1)
y = df['Flood Occurred']

# Handle Class Imbalance by Resampling
X_0 = X[y == 0]
X_1 = X[y == 1]
y_0 = y[y == 0]
y_1 = y[y == 1]

# Upsample minority class
X_1_upsampled, y_1_upsampled = resample(X_1,
                                         y_1,
                                         replace=True,
                                         n_samples=len(X_0), 
                                         random_state=42)

# Combine majority class with upsampled minority class
X_balanced = pd.concat([X_0, X_1_upsampled])
y_balanced = pd.concat([y_0, y_1_upsampled])

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# Train the Model
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
print("actual values:\n ",y_test.values)
print("predicted : \n",y_pred)

# Print Classification Report with zero_division handling
print(classification_report(y_test, y_pred, zero_division=0))

FileNotFoundError: [Errno 2] No such file or directory: 'flood_risk_dataset_india.csv'

In [ ]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Sample DataFrame (you can replace this with your actual CSV data)
data = pd.read_csv('flood_risk_dataset_india.csv')
df = pd.DataFrame(data)

# Haversine Function to calculate distance between two locations
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # Distance in km

# Predefined river level data for different locations
river_levels = {
    (17.385044, 78.486671): 85,  # Hyderabad
    (18.520407, 73.856255): 95,  # Pune
    (19.076090, 72.877426): 80,  # Mumbai
    (12.971599, 77.594566): 75   # Bangalore
}

# Simulate fetching the closest current river level based on user's location
def get_current_river_level(lat, lon):
    closest_distance = float('inf')  # Start with a very large distance
    closest_river_level = None
    
    # Iterate through predefined river level locations
    for (stored_lat, stored_lon), river_level in river_levels.items():
        distance = haversine(lat, lon, stored_lat, stored_lon)
        
        # Find the closest location
        if distance < closest_distance:
            closest_distance = distance
            closest_river_level = river_level
    
    return closest_river_level

# Predict Flood Risk Based on Location and Current River Level
def predict_flood_risk(new_lat, new_long, current_river_level, new_data):
    # Find the closest historical flood data
    closest_location = None
    min_distance = float('inf')
    threshold = None
    
    # Iterate through each historical flood location to find the closest one
    for index, row in new_data.iterrows():
        historical_lat, historical_long = row['Latitude'], row['Longitude']
        distance = haversine(new_lat, new_long, historical_lat, historical_long)
        
        # Find the closest historical flood data point
        if distance < min_distance:
            min_distance = distance
            closest_location = row
            threshold = row['Water Level (m)']  # Max historical river level for that location
    
    # If closest historical data is found, compare river levels
    if closest_location is not None:
        if current_river_level >= threshold:
            return f"Flood Risk: HIGH 🚨 (Closest Historical Location: {closest_location['Latitude']}, {closest_location['Longitude']})"
        else:
            return f"Flood Risk: LOW ✅ (Closest Historical Location: {closest_location['Latitude']}, {closest_location['Longitude']})"
    else:
        return "No historical flood data available for this location."

# Example: User's location (Hyderabad)
new_lat = 17.385044  # User's latitude
new_long = 78.486671  # User's longitude

# Get the closest river level
current_river_level = get_current_river_level(new_lat, new_long)

# Output the closest river level
print(f"Closest river level for the user's location is: {current_river_level} meters")

# Make prediction based on current river level and user's location
result = predict_flood_risk(new_lat, new_long, current_river_level, df)
print(result)


Closest river level for the user's location is: 85 meters
Flood Risk: HIGH 🚨 (Closest Historical Location: 17.42801644751927, 78.5064969511781)


In [ ]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

river_levels = {
    (17.385044, 78.486671): 85,  # Hyderabad (river level 85 meters)
    (18.520407, 73.856255): 95,  # Pune (river level 95 meters)
    (19.076090, 72.877426): 80,  # Mumbai (river level 80 meters)
    (12.971599, 77.594566): 75   # Bangalore (river level 75 meters)
}

roads = [
    ('A to B', (17.385044, 78.486671), (17.395044, 78.496671)),  # Road A to B (Hyderabad)
    ('B to C', (17.395044, 78.496671), (17.405044, 78.506671)),  # Road B to C (Hyderabad)
    ('A to C', (17.385044, 78.486671), (17.405044, 78.506671))   # Road A to C (Hyderabad)
]

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # Distance in km

def classify_road_flood_status(road_start, road_end, river_levels):
    flood_status = {'under_flood': False, 'near_flood': False, 'safe': False}

    # Calculate distance from road start and end points to flood-prone areas
    for river_location, river_level in river_levels.items():
        for point in [road_start, road_end]:
            distance = haversine(point[0], point[1], river_location[0], river_location[1])
            
            # Check if the road is under flood (within 200 meters of a flooded area)
            if distance < 0.2:  # 0.2 km = 200 meters
                flood_status['under_flood'] = True
            # Check if the road is near flood (within 500 meters of a flooded area)
            elif distance < 0.5:  # 0.5 km = 500 meters
                flood_status['near_flood'] = True
    
    # If no flood, consider it safe
    if not flood_status['under_flood'] and not flood_status['near_flood']:
        flood_status['safe'] = True
    
    return flood_status

# Function to check roads and classify their status
def check_road_status(roads, river_levels):
    road_status = []
    
    for road_name, road_start, road_end in roads:
        status = classify_road_flood_status(road_start, road_end, river_levels)
        road_status.append({
            'Road': road_name,
            'Status': status
        })
    
    return road_status

# Get the road status for all roads
road_status = check_road_status(roads, river_levels)

# Print the road status (whether they are under flood, near flood, or safe)
for status in road_status:
    print(f"Road: {status['Road']} | Status: {status['Status']}")


Road: A to B | Status: {'under_flood': True, 'near_flood': False, 'safe': False}
Road: B to C | Status: {'under_flood': False, 'near_flood': False, 'safe': True}
Road: A to C | Status: {'under_flood': True, 'near_flood': False, 'safe': False}
